<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Library-imports" data-toc-modified-id="Library-imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Library imports</a></span></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Feature Engineering</a></span><ul class="toc-item"><li><span><a href="#User-Logs-Features" data-toc-modified-id="User-Logs-Features-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>User Logs Features</a></span></li><li><span><a href="#Transactions-Features" data-toc-modified-id="Transactions-Features-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Transactions Features</a></span></li><li><span><a href="#Members-Features" data-toc-modified-id="Members-Features-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Members Features</a></span></li></ul></li><li><span><a href="#Merging-Features" data-toc-modified-id="Merging-Features-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Merging Features</a></span></li></ul></div>

# Introduction

[(Back to top)](#Table-of-Contents)

Feature engineering is a very integral part of the machine learning process, it comes after the data pre-processing and cleaning steps.

After having a clean data, we use it to generate features or input variables that the model will use to find correlations and patterns between these variables and the target variable that we want to predict.

The features in this use case are engineered relative to dates selected in the past, meaning that for each given date, features will only be calculated using the history prior to that date. And thus giving us multiple master tables or multiple snapshots of clients behaviors that vary with time and therefore giving the model more data to train.

# Library imports

[(Back to top)](#Table-of-Contents)

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import warnings
from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import dask.dataframe as dd
import os
import itertools
import matplotlib.ticker as ticker
import vaex
from dask.distributed import Client
from dask import delayed, compute
from datetime import datetime
from dateutil.relativedelta import relativedelta
from functools import reduce
from utils import *
alt.renderers.enable('default')
InteractiveShell.ast_node_interactivity = "all"
sns.set_theme(style="darkgrid")
warnings.filterwarnings('ignore')
sns.set_palette(sns.color_palette("Set3"))
client = Client(n_workers=6)


In [2]:
uid = 'msno'

In [3]:
# This can be changed to the directory where the datasets are stored
data_dir = './data'

# Training data for january, contains two columns : user id and binary churn target variable
train = pd.read_csv(os.path.join(data_dir, 'train.csv'), dtype={'is_churn': str})

train.head()

,msno,is_churn
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1


# Feature Engineering

## User Logs Features

[(Back to top)](#Table-of-Contents)

In this step of the feature engineering process, we will be using the user logs data to generate features containing information about the listening behavior of the clients.

Since the data is very large and cannot fit in the server's memory, I will be using the <a href="https://vaex.io/docs/api.html">vaex</a> library to be able to generate features for such large dataset.

I have previously used a python script to **split** the data (that has been provided in the `csv` format) into chunks with a format of `hdf5`.

In [9]:
df = vaex.open('./data/user_logs_split_*.hdf5')

In [10]:
# Giving the columns their proper names
names = ['msno','date','num_25','num_50','num_75','num_985','num_100','num_unq','total_secs']

for i, new_name in enumerate(names):
    df.rename(str(i), new_name)

'msno'

'date'

'num_25'

'num_50'

'num_75'

'num_985'

'num_100'

'num_unq'

'total_secs'

In [6]:
# The data contains a date column but that's in a format that cannot be handled by vaex
# we will need to convert from this format 20160301 to this format 2016-03-01 before converting to datetime type

df['year'] = df['date'] // 10000
df['month'] = (df['date'] // 100) % 100
df['day'] = df['date'] % 100

df['year'] = df['year'].astype(str)
df['month'] = df['month'].astype(str).str.zfill(2)
df['day'] = df['day'].astype(str).str.zfill(2)

df['date_formatted'] = df['year'] + '-' + df['month'] + '-' + df['day']
df['date_formatted'] = df['date_formatted'].astype('datetime64')

df.drop(['year', 'month', 'day'], inplace=True)

#,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,date_formatted
0,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150513,0,0,0,0,1,1,280.335,Timestamp('2015-05-13 00:00:00')
1,'rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=',20150709,9,1,0,0,7,11,1658.9479999999999,Timestamp('2015-07-09 00:00:00')
2,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150105,3,3,0,0,68,36,17364.956000000002,Timestamp('2015-01-05 00:00:00')
3,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150306,1,0,1,1,97,27,24667.317000000003,Timestamp('2015-03-06 00:00:00')
4,'yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=',20150501,3,0,0,0,38,38,9649.029,Timestamp('2015-05-01 00:00:00')
...,...,...,...,...,...,...,...,...,...,...
"391,865,090",'ccyBVC3HXqPi7D8GoxT0u3jntamqtPubA9hKhRQUsu0=',20150925,0,1,0,0,1,2,342.207,Timestamp('2015-09-25 00:00:00')
"391,865,091",'ccyBVC3HXqPi7D8GoxT0u3jntamqtPubA9hKhRQUsu0=',20160216,12,9,2,0,13,30,4220.209,Timestamp('2016-02-16 00:00:00')
"391,865,092",'ccyBVC3HXqPi7D8GoxT0u3jntamqtPubA9hKhRQUsu0=',20160515,15,9,10,5,15,47,7300.25,Timestamp('2016-05-15 00:00:00')
"391,865,093",'ccyBVC3HXqPi7D8GoxT0u3jntamqtPubA9hKhRQUsu0=',20160725,8,0,0,0,10,13,2498.097,Timestamp('2016-07-25 00:00:00')


In [7]:
def compute_aggregations_history(df, date_filter=20160201, resolution="D", resolution_value=30, list_aggs=None):
    """
    Filter df by only taking history prior to <date_filter> then generate aggregations each :
                                                                            <resolution_value> * <resolution>
    :param df: vaex dataframe containing raw data of clients (daily listening behavior of each client)
    :param date_filter: filter date, only take instances having a date inferior to this
    :param resolution: datetime frequency of aggregations
    :param resolution_value: number of units of <resolution> to take
    :param list_aggs: dictionary mapping the name of the output column to the aggregation to perform
    :return: pandas df containing aggregations performed
    """
    if list_aggs is None:
        list_aggs = []
    df_filtered = df[df.date <= date_filter]
    return df_filtered.groupby(['msno', vaex.BinnerTime(df_filtered['date_formatted'],
                                               resolution=resolution,
                                               every=resolution_value)]).agg(*list_aggs).to_pandas_df()


In [4]:
def aggregate_user_logs_history(df, date_filter):
    """
    Perform aggregations per client on their history prior to a given date.
    :param df: vaex dataframe containing raw data of clients (daily listening behavior of each client)
    :param date_filter: filter date, only take instances having a date inferior to this
    :return: pandas df containing aggregations performed
    """
    df_filtered = df[df.date < date_filter ]
    res = df_filtered.groupby(df_filtered.msno,
                          agg=[vaex.agg.sum('total_secs'), vaex.agg.mean('total_secs'), 
                               vaex.agg.std('total_secs'), vaex.agg.sum('num_unq'),
                               vaex.agg.mean('num_unq'), vaex.agg.std('num_unq'),
                               vaex.agg.count('msno')]).to_pandas_df()
    res['date_pred'] = date_filter // 100
    return res

In [8]:
# Testing the function created on a sample 
sample = df[:10000000]

In [9]:
# This is an example of calculating the features total seconds and average seconds of songs listened
# every 6 months for each client (this is only computed using the sample and not the whole dataset)
res = compute_aggregations_history(sample, 20160603, "M", 6, [{'total_seconds_6_months': vaex.agg.sum(sample.total_secs),
           'avg_seconds_6_months': vaex.agg.mean(sample.total_secs)}])

In [10]:
res.head(10)

,msno,date_formatted,total_seconds_6_months,avg_seconds_6_months
0,V5gendGLIUHUEI9hglLiWprbX62M9dVKxHz6Nqr6rYk=,2015-06-01,3204.518,3204.518
1,V5gendGLIUHUEI9hglLiWprbX62M9dVKxHz6Nqr6rYk=,2015-12-01,1396.129,1396.129
2,w5TmRNc/J/33Oz+x4Ur23N1tCgqsC/It4u83vW5rpEw=,2014-12-01,206.207,206.207
3,uWz9ubBCh0Y4klpCgPZtstkgF9CRWOq1JAvEuoldU40=,2014-12-01,3745.980,3745.980
4,uWz9ubBCh0Y4klpCgPZtstkgF9CRWOq1JAvEuoldU40=,2015-06-01,6098.470,3049.235
5,uWz9ubBCh0Y4klpCgPZtstkgF9CRWOq1JAvEuoldU40=,2015-12-01,48261.662,24130.831
6,ZU/wGdIJSUGkv0v85/dFboKnAxyqWL/XYbbBXDRxU9c=,2014-12-01,942.345,942.345
7,VsFHZOZyDS5JWMU9vrVAUkKiyOaj9hxwaokAkLtHiek=,2014-12-01,1056.457,1056.457
8,8xXFFiPwv1phcRn1d0ZFKpk8yBaAtdlm+WMlu9r0P0U=,2014-12-01,1147.958,1147.958
9,3ZOrqDxhkzWY7RkqxjPPSW8UuNd1OVUfLftWp4iVM1k=,2014-12-01,19276.360,19276.360


We can from the previous result, that for each client we get multiple instances (relative to multiple dates distanced by 6 months) that gives the total and average of seconds of songs that each user has listened to during the previous 6 months.

We can also the user logs dataset to get the number of logins for each user, this can be a very important information for the model since inactive clients tend to churn more likely than active clients.

Since the dataset is very large, I have tested multiple values for aggregations of number of logins of clients and have found that the monthly aggregations fit in memory.

In [5]:
def last_login_days(user_logs, date):
    """
    Compute the number of days for each client relative to a given date.
    :param user_logs: vaex dataframe containing raw data of clients (daily listening behavior of each client)
    :param date: filter date, only take instances having a date inferior to this
    :return: pandas df containing aggregations performed
    """
    user_logs_filtered = user_logs[user_logs.date < date * 100 + 1]
    res = user_logs_filtered.groupby(uid, agg=[vaex.agg.max('date')]).to_pandas_df()
    res['last_login_days'] = (datetime(date // 100, date % 100, 1) - pd.to_datetime(res['date_max'], format='%Y%m%d')).dt.days
    res['date_pred'] = date
    return res.drop('date_max', 1)

In [11]:
# computing the number of logins in each month for every client
res = df.groupby(['msno', vaex.BinnerTime(df['date_formatted'],
                                               resolution="M",
                                               every=1)]).agg({'nbr_logins_monthly' : vaex.agg.count(df.date)}).to_pandas_df()

In [12]:
res.head()

,msno,date_formatted,nbr_logins_monthly
0,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,2015-03-01,6
1,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,2015-04-01,29
2,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,2015-05-01,27
3,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,2015-06-01,26
4,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,2015-07-01,28


In [ ]:
res.to_csv('nbr_logins_monthly.csv', index=False)

In [6]:
nbr_logins = pd.read_csv('nbr_logins_monthly.csv')

nbr_logins['date_formatted'] = pd.to_datetime(nbr_logins['date_formatted'], format='%Y-%m-%d')

In [7]:
date = 201603

In [8]:
def months_last_login(unstacked_df, date):
    """
    Generates a dataframe that contains the number of months since last login, relative to a given date.
    """
    temp = unstacked_df[[x for x in unstacked_df.columns if x < pd.to_datetime(date, format='%Y%m')]]
    temp = temp.isnull()
    res = temp.apply(
        lambda x: x.where(x).last_valid_index(), axis=1).reset_index(name="date_last_login")
    res.date_last_login.fillna(datetime(2015, 1, 1), inplace=True)
    res['months_since_login'] = (pd.to_datetime(date, format='%Y%m') - res['date_last_login']).dt.days / 30
    res['date_pred'] = date
    return res[[uid, 'date_pred', "months_since_login"]]

In [57]:
# the following function will be useful later for parallel execution of feature engineering relative to 
# multiple dates
months_last_login_dates = lambda unstacked_df, dates: [months_last_login(unstacked_df, date) for date in dates]

In [19]:
unstacked_df = nbr_logins.groupby(
    [uid, "date_formatted"])[
    'nbr_logins_monthly'].mean().unstack('date_formatted')

In [9]:
def stats_logins_monthly(df, date):
    """
    Compute statistics of monthly listening behavior of ever client prior to a certain date.
    :param df: vaex dataframe containing raw data of clients (daily listening behavior of each client)
    :param date: filter date, only take instances having a date inferior to this
    :return: pandas df containing aggregations performed
    """
    df_filtered = df[df.date_formatted < datetime(date // 100, date % 100, 1)]
    
    stats_logins = df.groupby(uid)['nbr_logins_monthly'].agg(['mean', 'max', 'min', 'std', 'count'])
    stats_logins.columns = ['_'.join(['nbr_logins_monthly', x]) for x in stats_logins.columns]

    stats_logins = stats_logins.reset_index()
    stats_logins.fillna(0, inplace=True)
    stats_logins['date_pred'] = date
    return stats_logins

In [30]:
stats_logins_monthly(nbr_logins, 201601)

,msno,nbr_logins_monthly_mean,nbr_logins_monthly_max,nbr_logins_monthly_min,nbr_logins_monthly_std,nbr_logins_monthly_count,date_pred
0,+++4vcS9aMH7KWdfh5git6nA5fC5jjisd5H/NcM++WM=,1.000000,1,1,0.000000,1,201601
1,+++EI4HgyhgcJHIPXk/VRP7bt17+2joG39T6oEfJ+tc=,1.000000,1,1,0.000000,1,201601
2,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,7.000000,7,7,0.000000,1,201601
3,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,23.923077,31,8,5.992816,26,201601
4,+++TipL0Kt3JvgNE9ahuJ8o+drJAnQINtxD4c5GePXI=,1.000000,1,1,0.000000,1,201601
...,...,...,...,...,...,...,...
5234029,zzzeSzWGUrQw+eP47oa1CXqL/im1Uq6/JYAJs8oGjI8=,1.000000,1,1,0.000000,1,201601
5234030,zzzqx+aMPSFYjW71JqJ6T/hita+iVemVWzJTE4yQRx8=,2.000000,2,2,0.000000,1,201601
5234031,zzztPAN9xjMytpZ0RN2gU9mScDULJnHQZK8eZb4uELU=,6.000000,10,2,3.265986,4,201601
5234032,zzztsqkufVj9DPVJDM3FxDkhlbCL5z4aiYxgPSGkIK4=,3.750000,9,1,3.593976,4,201601


In [22]:
dates = [201603, 201608]

In [10]:
def distribute_processes(func, **kwargs):   
    """
    Splits the process execution across multiple cores, useful for parallel execution of feature engineering
    relative to multiple dates
    :param func: the function to be parallelized
    :param kwargs: keyword arguments for the function
    :return: final pandas dataframe containing the joined dataframes after the parallel computing
    """
    delayed_results = delayed(func)(**kwargs)
    results = compute(delayed_results, scheduler="processes")
    final_res = pd.concat(results[0])
    
    return final_res

The previous function is very useful for reducing the amount of time required to generate the features (that are computed relative to multiple dates), and will be later implemented in a function that merges them all.

In [27]:
distribute_processes(months_last_login_dates, unstacked_df=unstacked_df.head(500), dates=dates)

,msno,date_pred,months_since_login
0,+++4vcS9aMH7KWdfh5git6nA5fC5jjisd5H/NcM++WM=,201603,0.966667
1,+++EI4HgyhgcJHIPXk/VRP7bt17+2joG39T6oEfJ+tc=,201603,0.966667
2,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,201603,0.966667
3,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,201603,14.166667
4,+++TipL0Kt3JvgNE9ahuJ8o+drJAnQINtxD4c5GePXI=,201603,0.966667
...,...,...,...
495,++OrO+qyw4KsyivgnrEzl5ALtN7gj7BNEodVdR5UIss=,201608,1.033333
496,++OtuWs8cKG2710CCnsZTeVKK9Co38gFamjS9nkus+8=,201608,1.033333
497,++OvJH5FmfZ5CRrYfmbQEk7tJwCZhsJnkWbxClRaUpw=,201608,6.066667
498,++OwfFddNO06garROtSdQUfkCwbKF+9pfndYHGRKDas=,201608,1.033333


## Transactions Features

[(Back to top)](#Table-of-Contents)

In [11]:
transactions = pd.read_csv(os.path.join(data_dir, 'transactions_cleaned_final.csv'))

In [12]:
date = 201603

In [13]:
transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,diff_dates
0,JQqsvSC2BNWif2jSUS0BJ3JT1/oREqxrrL4jNjbjFWU=,41,30,149,149,1,2015-01-01,20150202,0,NaN
1,Z1CV2CrBP9tR4Xr5H4VqgGzx650b2HtwJWJV/ExQNaw=,41,30,149,149,1,2015-01-01,20150201,0,NaN
2,flo7XEv3URucBlewH+Y/QgdmrAapKq3+NMs97Z+A9Sc=,41,30,149,149,1,2015-01-01,20150201,0,NaN
3,Bv0w1EeVjiPLB8r5mR1ny2x8guIdqzGNpKa1LgJ1d9M=,41,30,149,149,1,2015-01-01,20150203,0,NaN
4,fSn3mhLjNEREk1f/84iU1FFas2l4zAaMk4JT15WLlvE=,41,30,149,149,1,2015-01-01,20150201,0,NaN


In [14]:
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'], format='%Y-%m-%d')

In [43]:
def agg_trx(transactions, date, aggs={}):
    """
    Compute aggregations of features related to transactions for every client prior to a certain date
    :param transactions: the transactions dataframe
    :param date: filter date, only take instances having a date inferior to this date
    :param aggs: a list containing the aggregations to perform (a list of dictionaries)
    :return: pandas df containing aggregations performed 
    """
    filtered_transactions = transactions[transactions.transaction_date < datetime(date // 100, date % 100, 1)]
    aggs_trx = filtered_transactions.groupby(uid).agg(aggs)
    aggs_trx.columns = ['_'.join(x) for x in aggs_trx.columns.ravel()]
    aggs_trx.reset_index(inplace=True)
    
    aggs_trx['date_pred'] = date
    return aggs_trx

In [9]:
agg_trx(transactions, date, aggs={'actual_amount_paid': ['min', 'max', 'mean', 'std', 'sum'],
                                  'is_auto_renew': ['mean', 'sum'],
                                 'is_cancel': ['sum', 'mean'], 
                                  'msno': 'count'})

,msno,actual_amount_paid_min,actual_amount_paid_max,actual_amount_paid_mean,actual_amount_paid_std,actual_amount_paid_sum,is_auto_renew_mean,is_auto_renew_sum,is_cancel_sum,is_cancel_mean,msno_count
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,1788,1788,1788.000000,NaN,1788,0.000000,0,0,0.000000,1
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,149,149,149.000000,0.000000,1788,1.000000,12,0,0.000000,12
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,149,149,149.000000,0.000000,2086,1.000000,14,0,0.000000,14
3,++/Gw1B9K+XOlB3hLTloeUK2QlCa2m+BJ8TrzGf7djI=,149,894,231.777778,248.333333,2086,0.888889,8,1,0.111111,9
4,++/TR7WI15q2ZCtOXmoap7jR+kEhbMVE5swOqsfqpqI=,149,149,149.000000,0.000000,1341,1.000000,9,1,0.111111,9
...,...,...,...,...,...,...,...,...,...,...,...
1639823,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,119,149,125.428571,12.774459,1756,1.000000,14,0,0.000000,14
1639824,zzz672Xpk1wKox75rJ5gak43ZkFQUV1f7Xek3jnPeRM=,0,0,0.000000,NaN,0,0.000000,0,0,0.000000,1
1639825,zzz9+ZF4+GMyt63oU8xfjo1EkvRqH5OINlES0RUJI6I=,129,129,129.000000,0.000000,645,1.000000,5,0,0.000000,5
1639826,zzztsqkufVj9DPVJDM3FxDkhlbCL5z4aiYxgPSGkIK4=,0,0,0.000000,NaN,0,0.000000,0,0,0.000000,1


In [100]:
res = transactions.groupby(uid)['transaction_date'].diff()

In [101]:
transactions['diff_dates'] = res.dt.days

The column `diff_dates` contains the **difference in days** between transactions performed by each client.

This feature should be very useful for the model, since it will give great insights for the model about the frequency of subscriptions of the client.

In [112]:
transactions.to_csv('./data/transactions_cleaned_final.csv', index=False)

In [41]:
transactions_cleaned = transactions.dropna(subset=['diff_dates'])

In [16]:
def stats_diff_dates(trx_cleaned, date):
    """
    Compute statistics for the frequency of subscriptions.
    :param trx_cleaned: cleaned transactions dataframe
    :param date: filter date, only take instances having a date inferior to this date
    :return: pandas df containing aggregations performed 
    """
    trx_cleaned_filtered = trx_cleaned[trx_cleaned.transaction_date < datetime(date // 100, date % 100, 1)]
    
    res_diff_dates = trx_cleaned_filtered.groupby(uid)['diff_dates'].agg(['min', 'max', 'mean', 'std'])
    res_diff_dates.columns = ['_'.join(['diff_dates', x]) for x in res_diff_dates.columns]
    res_diff_dates.reset_index(inplace=True)
    
    res_diff_dates['date_pred'] = date
    return res_diff_dates

In [59]:
stats_diff_dates(transactions_cleaned, date)

,msno,diff_dates_min,diff_dates_max,diff_dates_mean,diff_dates_std,date_pred
0,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,28.0,60.0,33.181818,8.975725,201603
1,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,28.0,31.0,30.461538,0.877058,201603
2,++/Gw1B9K+XOlB3hLTloeUK2QlCa2m+BJ8TrzGf7djI=,8.0,31.0,27.250000,7.851297,201603
3,++/TR7WI15q2ZCtOXmoap7jR+kEhbMVE5swOqsfqpqI=,1.0,31.0,26.750000,10.430039,201603
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,28.0,60.0,33.181818,8.975725,201603
...,...,...,...,...,...,...
1267967,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,0.0,31.0,27.090909,9.278519,201603
1267968,zzyuTnRVQQGxs0UN8qzU2Pedh3z8F7UeoiQpNhJRomM=,30.0,31.0,30.500000,0.547723,201603
1267969,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,7.0,154.0,44.000000,47.056197,201603
1267970,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,28.0,31.0,30.461538,0.877058,201603


In [17]:
# parsing through every training data set, each training data set contains three columns: the column <uid>
# the column <is_churn> and the <date_prediction> column
paths_churn_data = [x for x in os.listdir('data') if x.startswith('train_set')]

In [18]:
churn_dfs = [pd.read_csv(os.path.join('data', x)) for x in paths_churn_data]

In [19]:
history_churn_clients = pd.concat(churn_dfs)

In [20]:
history_churn_clients['is_churn'] = history_churn_clients['is_churn'].astype(int)

history_churn_clients['date_prediction'] = pd.to_datetime(history_churn_clients['date_prediction'], format='%Y%m%d')

In [21]:
def stats_churn(history_churn_clients, date):
    """
    Compute aggregations of features related to the churn history.
    :param history_churn_clients: churn history dataframe
    :param date: filter date, only take instances having a date inferior to this date
    :return: pandas df containing aggregations performed 
    """
    history_filtered = history_churn_clients[history_churn_clients.date_prediction < datetime(date // 100, date % 100, 1)]
    res_stats_churn = history_filtered.groupby(uid)['is_churn'].agg(['mean', 'sum'])
    res_stats_churn.columns = ['_'.join(['churn', x]) for x in res_stats_churn.columns]
    
    res_stats_churn.reset_index(inplace=True)
    res_stats_churn['date_pred'] = date
    
    return res_stats_churn

In [94]:
stats_churn(history_churn_clients, date)

,msno,churn_mean,churn_sum,date_pred
0,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0.5,1,201603
1,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,0.0,0,201603
2,++/Gw1B9K+XOlB3hLTloeUK2QlCa2m+BJ8TrzGf7djI=,1.0,1,201603
3,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,0.5,1,201603
4,++/gTmVgKUbNFmsTiriZdWV1uZIrLXCUiEWN0fEU6BM=,0.0,0,201603
...,...,...,...,...
968145,zzyuTnRVQQGxs0UN8qzU2Pedh3z8F7UeoiQpNhJRomM=,0.0,0,201603
968146,zzz/5bh+8EOZTpIRKWiiaAwEH4uYwfDUaiahJlvABiQ=,0.0,0,201603
968147,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,0.5,1,201603
968148,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,0.0,0,201603


In [22]:
def days_last_trx(transactions, date):
    """
    Compute the number of days since the last transaction (subscription) made each client.
    :param transactions: transactions dataframe
    :param date: filter date, only take instances having a date inferior to this date
    :return: pandas df containing the number of days since last transaction
    """    
    transactions_filtered = transactions[transactions.transaction_date < datetime(date // 100, date % 100, 1)]
    res_last_trx = transactions_filtered.groupby(uid)['transaction_date'].max().reset_index(name="last_trx_date")
    res_last_trx['date_pred'] = date
    res_last_trx['days_last_trx'] = (datetime(date // 100, date % 100, 1) - res_last_trx['last_trx_date']).dt.days
    
    return res_last_trx[[uid, 'date_pred', 'days_last_trx']]

In [100]:
days_last_trx(transactions, date)

,msno,date_pred,days_last_trx
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,201603,101
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,201603,30
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,201603,4
3,++/Gw1B9K+XOlB3hLTloeUK2QlCa2m+BJ8TrzGf7djI=,201603,195
4,++/TR7WI15q2ZCtOXmoap7jR+kEhbMVE5swOqsfqpqI=,201603,188
...,...,...,...
1639823,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,201603,26
1639824,zzz672Xpk1wKox75rJ5gak43ZkFQUV1f7Xek3jnPeRM=,201603,270
1639825,zzz9+ZF4+GMyt63oU8xfjo1EkvRqH5OINlES0RUJI6I=,201603,17
1639826,zzztsqkufVj9DPVJDM3FxDkhlbCL5z4aiYxgPSGkIK4=,201603,267


## Members Features

[(Back to top)](#Table-of-Contents)

In [23]:
members = pd.read_csv(os.path.join('data', 'members_cleaned.csv'))

In [24]:
members.drop('Unnamed: 0', axis=1, inplace=True)
members['registration_init_time'] = pd.to_datetime(members['registration_init_time'], format='%Y%m%d')
members['gender'] = members['gender'].astype("category").cat.codes

In [11]:
members.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,27.0,1,11,2011-09-11
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,27.0,1,7,2011-09-14
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,27.0,1,11,2011-09-15
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,27.0,1,11,2011-09-15
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32.0,0,9,2011-09-15


In [31]:
members.dtypes

msno                              object
city                               int64
bd                               float64
gender                              int8
registered_via                     int64
registration_init_time    datetime64[ns]
dtype: object

In [2]:
def dummify_column_top_5(members, column):
    """
    Perform one hot encoding and only select the top 5 (most recurring) values
    :param members: members dataframe
    :param column: the categorical column to which we apply the dummyfying
    :return: pandas dataframe
    """    
    top_5_column = members[column].value_counts().reset_index().loc[:5, 'index'].tolist()
    members.loc[~members[column].isin(top_5_column), column] = 'other'
    dummies_column = pd.get_dummies(members[column])
    dummies_column.columns = ['_'.join([column, str(x)]) for x in dummies_column.columns]    
    return dummies_column

In [33]:
dummify_column_top_5(members, 'city')

,city_1,city_4,city_5,city_13,city_15,city_22,city_other
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0
4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...
6769468,1,0,0,0,0,0,0
6769469,1,0,0,0,0,0,0
6769470,0,0,0,0,1,0,0
6769471,1,0,0,0,0,0,0


In [26]:
def members_static_features(members, date):
    """
    Select static features for each client
    :param members: members dataframe
    :param date: filter date, only take instances having a date inferior to this date
    :return: pandas dataframe
    """
    static_features = members[[uid, 'bd', 'gender']]
    dummies_city  = dummify_column_top_5(members, 'city')
    dummies_registration  = dummify_column_top_5(members, 'registered_via')
    
    final_res = pd.concat([static_features, dummies_city, dummies_registration], axis=1)
    final_res['date_pred'] = date
    
    return final_res

In [35]:
members_static_features(members)

,msno,bd,gender,city_1,city_4,city_5,city_13,city_22,city_other,registered_via_3,registered_via_4,registered_via_7,registered_via_9,registered_via_11,registered_via_13,registered_via_other
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,27.0,1,1,0,0,0,0,0,0,0,0,0,1,0,0
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,27.0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,27.0,1,1,0,0,0,0,0,0,0,0,0,1,0,0
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,27.0,1,1,0,0,0,0,0,0,0,0,0,1,0,0
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,32.0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6769468,VSGkb3hyBRUtb/b1MQUZbvOkktS3vKLnhMHW0CF8eyU=,27.0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
6769469,nWjH7glPkZ7jOVaCRwwjlpmp0T1hSWdv8hMJxiWCwKc=,27.0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
6769470,GH+b5+1tlv7ZZXsA8upBzVXMTLyffKcsF7WoU8b5rOI=,26.0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
6769471,XVlwT3fdCFGKqerEKBzUIjK+jzI6jzSke4cDMVhYyjE=,27.0,1,1,0,0,0,0,0,0,1,0,0,0,0,0


In [27]:
def compute_tenure(members, date):
    """
    Compute the number of days since the client started being a member.
    :param members: members dataframe
    :param date: filter date, only take instances having a date inferior to this date
    :return: pandas dataframe
    """
    members['tenure_days'] = (datetime(date // 100, date % 100, 1) - members['registration_init_time']).dt.days
    members['date_pred'] = date
    
    return members[[uid, 'tenure_days', "date_pred"]]

In [74]:
compute_tenure(members, date)

,msno,tenure_days,date_pred
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1633,201603
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1630,201603
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1629,201603
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1629,201603
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,1629,201603
...,...,...,...
6769468,VSGkb3hyBRUtb/b1MQUZbvOkktS3vKLnhMHW0CF8eyU=,133,201603
6769469,nWjH7glPkZ7jOVaCRwwjlpmp0T1hSWdv8hMJxiWCwKc=,133,201603
6769470,GH+b5+1tlv7ZZXsA8upBzVXMTLyffKcsF7WoU8b5rOI=,133,201603
6769471,XVlwT3fdCFGKqerEKBzUIjK+jzI6jzSke4cDMVhYyjE=,133,201603


# Merging Features

[(Back to top)](#Table-of-Contents)

In [4]:
train_paths = sorted([os.path.join('./data', x) for x in os.listdir('data') if x.startswith('train_set')])

In [5]:
train_paths

['./data/train_set20151231.csv',
 './data/train_set20160131.csv',
 './data/train_set20160229.csv',
 './data/train_set20160331.csv',
 './data/train_set20160430.csv',
 './data/train_set20160531.csv',
 './data/train_set20160630.csv',
 './data/train_set20160731.csv',
 './data/train_set20160831.csv',
 './data/train_set20160930.csv',
 './data/train_set20161031.csv',
 './data/train_set20161130.csv',
 './data/train_set20161231.csv',
 './data/train_set20170131.csv',
 './data/train_set20170228.csv',
 './data/train_set20170331.csv']

In [6]:
def train_data(date):
    """
    Generate a dataframe containing for each client a Flag 0/1 if the client will churn the following month 
    relative to a date. This dataset will be used as a "reference", since it will be the first one before
    left merging with the other dataframes generated by building the features.
    """
    history_cutoff = pd.to_datetime(date, format='%Y%m') - relativedelta(days=1)
    train_set_path = 'train_set' + str(history_cutoff.year * 10000 + history_cutoff.month * 100 + history_cutoff.day) + '.csv'
    train_df = pd.read_csv(os.path.join('./data', train_set_path))
    train_df['date_pred'] = date
    try: 
        train_df.drop('date_prediction', axis=1, inplace=True)
    except:
        pass
    train_df['is_churn'] = train_df['is_churn'].astype(int)
    return train_df

In [7]:
train_data(201702)

,msno,is_churn,date_pred
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,201702
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,201702
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,201702
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,201702
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,201702
...,...,...,...
992926,tUM0yxAj50Vc35vQZ++xMlomdyeLJUW9lEcoPnt3H+g=,0,201702
992927,KQS8etmfGgvE/7Y9gK+E9wdLnRTI0lyyPXaXL3I8E4c=,0,201702
992928,8/jDLgNREuWI9hcKVYp8723nmavn01T+AuMWkK3uM7g=,0,201702
992929,iNV99F1Rml7EMndOeLI0Y/iek6aCj/Qp1Z4dZvR+sak=,0,201702


In [8]:
members = pd.read_csv(os.path.join('data', 'members_cleaned.csv'))

In [9]:
members.drop('Unnamed: 0', axis=1, inplace=True)
members['registration_init_time'] = pd.to_datetime(members['registration_init_time'], format='%Y%m%d')
members['gender'] = members['gender'].astype("category").cat.codes

In [10]:
nbr_logins = pd.read_csv('nbr_logins_monthly.csv')

nbr_logins['date_formatted'] = pd.to_datetime(nbr_logins['date_formatted'], format='%Y-%m-%d')

In [11]:
paths_churn_data = [x for x in os.listdir('data') if x.startswith('train_set')]

churn_dfs = [pd.read_csv(os.path.join('data', x)) for x in paths_churn_data]

history_churn_clients = pd.concat(churn_dfs)

history_churn_clients['is_churn'] =history_churn_clients['is_churn'].astype(int)

history_churn_clients['date_prediction'] = pd.to_datetime(history_churn_clients['date_prediction'], format='%Y%m%d')

In [12]:
transactions = pd.read_csv(os.path.join(data_dir, 'transactions_cleaned_final.csv'))

In [13]:
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'], format='%Y-%m-%d')

In [14]:
transactions_cleaned = transactions.dropna(subset=['diff_dates'])

In [15]:
user_logs = vaex.open('./data/user_logs_split_*.hdf5')

In [16]:
names = ['msno','date','num_25','num_50','num_75','num_985','num_100','num_unq','total_secs']

for i, new_name in enumerate(names):
    user_logs.rename(str(i), new_name)

'msno'

'date'

'num_25'

'num_50'

'num_75'

'num_985'

'num_100'

'num_unq'

'total_secs'

In [17]:
uid = "msno"

In [18]:
history_churn_clients

,msno,is_churn,date_prediction
0,++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,0,2016-09-30
1,+/HS8LzrRGXolKbxRzDLqrmwuXqPOYixBIPXkyNcKNI=,0,2016-09-30
2,+/namlXq+u3izRjHCFJV4MgqcXcLidZYszVsROOq/y4=,0,2016-09-30
3,+0/X9tkmyHyet9X80G6GTrDFHnJqvai8d1ZPhayT0os=,0,2016-09-30
4,+09YGn842g6h2EZUXe0VWeC4bBoCbDGfUboitc0vIHw=,0,2016-09-30
...,...,...,...
880719,ybSNUcJTHwuKAbs1wSbFrjkqRnk15vzIrryj2EfZlX4=,1,2016-11-30
880720,yrjWpxd2aX3gFdU7gBADbuB2eAVt3FSY0SgwOM82lmE=,1,2016-11-30
880721,z4v8bvd2GRRo4QYHluYY4iyus4cAuoKEj64MPOgDOzU=,1,2016-11-30
880722,z9CSElpjKfTDWmwBIRla8dCGAWNzQZmvR41qgERkxac=,1,2016-11-30


In [19]:
def merge_all_features(date):
    """
    Construct all features prior to a certain date and join them in a single master table.
    :param date: filter date, only take instances having a date inferior to this date
    :return: pandas dataframe containing merged features
    """
    print(date)
    df0 = train_data(date)
    
    df1 = members_static_features(members, date)

    df2 = compute_tenure(members, date)

    df3 = agg_trx(transactions, date, aggs={'actual_amount_paid': ['min', 'max', 'mean', 'std', 'sum'],
                                      'is_auto_renew': ['mean', 'sum'],
                                     'is_cancel': ['sum', 'mean'], 
                                      'msno': 'count'})

    df4 = stats_diff_dates(transactions_cleaned, date)

    df5 = stats_churn(history_churn_clients, date)

    df6 = days_last_trx(transactions, date)

    df7 = aggregate_user_logs_history(user_logs, date * 100  + 1)
    
    df8 = stats_logins_monthly(nbr_logins, date)
    
    df9 = last_login_days(user_logs, date)
    
    final_df = reduce(lambda left, right: left.merge(right, how="left", on=[uid, "date_pred"]), 
                      [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9])
    
    return final_df

In [20]:
# list of dates relative to which the features will be constructed
dates_preds = [201702, 201703, 201704]

In [21]:
dfMasterTable = pd.concat([merge_all_features(date) for date in dates_preds])

In [23]:
dfMasterTable.to_csv('dfMasterTable.csv', index=False)

In [7]:
dfMasterTable.head(5)

,msno,is_churn,date_pred,bd,gender,city_1,city_4,city_5,city_13,city_15,city_22,city_other,registered_via_3,registered_via_4,registered_via_7,registered_via_9,registered_via_11,registered_via_13,registered_via_other,tenure_days,actual_amount_paid_min,actual_amount_paid_max,actual_amount_paid_mean,actual_amount_paid_std,actual_amount_paid_sum,is_auto_renew_mean,is_auto_renew_sum,is_cancel_sum,is_cancel_mean,msno_count_x,diff_dates_min,diff_dates_max,diff_dates_mean,diff_dates_std,churn_mean,churn_sum,days_last_trx,total_secs_sum,total_secs_mean,total_secs_std,num_unq_sum,num_unq_mean,num_unq_std,msno_count_y,nbr_logins_monthly_mean,nbr_logins_monthly_max,nbr_logins_monthly_min,nbr_logins_monthly_std,nbr_logins_monthly_count,last_login_days
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,201702,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4319.0,0.0,149.0,74.500000,105.358910,149.0,0.000000,0.0,0.0,0.000000,2.0,68.0,68.0,68.000000,NaN,1.000000,1.0,25.0,9.227921e+04,4613.960400,4240.848005,394.0,19.700000,16.012807,20.0,8.666667,18.0,2.0,8.326664,3.0,3.0
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,201702,38.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4318.0,149.0,180.0,150.409091,6.609222,3309.0,0.954545,21.0,1.0,0.045455,22.0,16.0,95.0,34.809524,16.095400,0.250000,1.0,1.0,2.614515e+06,5027.913888,5564.554567,7229.0,13.901923,14.006454,520.0,20.840000,31.0,1.0,9.485427,25.0,33.0
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,201702,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4126.0,149.0,150.0,149.200000,0.421637,1492.0,0.800000,8.0,1.0,0.100000,10.0,12.0,118.0,38.333333,30.512293,0.333333,1.0,20.0,2.597385e+06,10959.428928,8172.306431,11533.0,48.662447,35.829472,237.0,19.750000,30.0,4.0,8.945542,12.0,1.0
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,201702,23.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4109.0,0.0,1788.0,894.000000,1264.306925,1788.0,0.000000,0.0,0.0,0.000000,2.0,2.0,2.0,2.000000,NaN,NaN,NaN,392.0,4.840589e+06,6779.536436,6303.076898,17943.0,25.130252,24.978054,714.0,28.269231,31.0,21.0,3.244166,26.0,2.0
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,201702,27.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4053.0,149.0,1788.0,447.000000,601.297882,3576.0,0.000000,0.0,0.0,0.000000,8.0,3.0,409.0,108.142857,151.258530,0.000000,0.0,4.0,1.837821e+07,25038.428218,11939.337626,71565.0,97.500000,45.000447,734.0,29.153846,31.0,21.0,2.460769,26.0,2.0


In [6]:
dfMasterTable.columns

Index(['msno', 'is_churn', 'date_pred', 'bd', 'gender', 'city_1', 'city_4',
       'city_5', 'city_13', 'city_15', 'city_22', 'city_other',
       'registered_via_3', 'registered_via_4', 'registered_via_7',
       'registered_via_9', 'registered_via_11', 'registered_via_13',
       'registered_via_other', 'tenure_days', 'actual_amount_paid_min',
       'actual_amount_paid_max', 'actual_amount_paid_mean',
       'actual_amount_paid_std', 'actual_amount_paid_sum',
       'is_auto_renew_mean', 'is_auto_renew_sum', 'is_cancel_sum',
       'is_cancel_mean', 'msno_count_x', 'diff_dates_min', 'diff_dates_max',
       'diff_dates_mean', 'diff_dates_std', 'churn_mean', 'churn_sum',
       'days_last_trx', 'total_secs_sum', 'total_secs_mean', 'total_secs_std',
       'num_unq_sum', 'num_unq_mean', 'num_unq_std', 'msno_count_y',
       'nbr_logins_monthly_mean', 'nbr_logins_monthly_max',
       'nbr_logins_monthly_min', 'nbr_logins_monthly_std',
       'nbr_logins_monthly_count', 'last_login_days

The final master table contains multiple kinds of features that are constructed using the provided raw data of the clients of the music streaming service company. They are listed as follows:

- Static features (using the `members` dataset) that don't vary through time i.e: **Gender**, **City**, **Registration method** ...

- Statistics of listeting behavior (using the `transactions` dataset) : by aggregating the history of transactions for each client and computing the **statistics of the amount paid**, the **number of months on auto renew**, the **history of cancels**, **days since last transaction** and **statistics of time difference between transactions**.

- Statistics of the login behavior (using the `user_logs` dataset) : by aggregation the daily listening behavior for each client and computing the **statistics of total seconds** spent on the app, the **statistics of the the songs listened to**, **last login in days** and **tenure**.

This data should be cleaned further by **filling the null values** and **getting rid of the outliers** before plugging it to the model. **Getting rid of highly correlated features** is also necessary as well as doing proper **Feature Selection**.